In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
# Load the dataset
def load_data():
    X_train = pd.read_csv('HADDataset/train/X_train.txt', delim_whitespace=True, header=None)
    y_train = pd.read_csv('HADDataset/train/y_train.txt', delim_whitespace=True, header=None)
    X_test = pd.read_csv('HADDataset/test/X_test.txt', delim_whitespace=True, header=None)
    y_test = pd.read_csv('HADDataset/test/y_test.txt', delim_whitespace=True, header=None)

    return (X_train, y_train), (X_test, y_test)

In [3]:
# Preprocess the data
def preprocess_data(X, y):
    X = (X - X.mean()) / X.std()  # Normalize the data
    le = LabelEncoder()
    y_encoded = le.fit_transform(y.values.ravel())  # Encode labels
    y_categorical = to_categorical(y_encoded)  # One-hot encode labels
    return X, y_categorical

# Reshape the data for LSTM input
def reshape_data(X, y, timesteps):
    Xs, ys = [], []
    for i in range(len(X) - timesteps):
        Xs.append(X.iloc[i:i + timesteps].values)
        ys.append(y[i + timesteps])
    return np.array(Xs), np.array(ys)